## Installation

1. If you haven't already installed Python3, get it from [Python.org](https://www.python.org/downloads/)
1. If you haven't already installed Jupyter Notebook, run `python3 -m pip install jupyter`
1. In Terminal, cd to the folder in which you downloaded this file and run `jupyter notebook`. This should open up a page in your web browser that shows all of the files in the current directory, so that you can open this file. You will need to leave this Terminal window up and running and use a different one for the rest of the instructions.
1. If you didn't install keras previously, install it now
    1. Install the tensorflow machine learning library by typing the following into Terminal:
    `pip3 install --upgrade tensorflow`
    1. Install the keras machine learning library by typing the following into Terminal:
    `pip3 install keras`


## Documentation/Sources
* [https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/) for information on sequence classification with keras
* [https://keras.io/](https://keras.io/) Keras API documentation
* [Keras recurrent tutorial](https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent)

## The IMDB Dataset
The [IMDB dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) consists of movie reviews (x_train) that have been marked as positive or negative (y_train). See the [Word Vectors Tutorial](https://github.com/jennselby/MachineLearningTutorials/blob/master/WordVectors.ipynb) for more details on the IMDB dataset.

In [2]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()

For a standard keras model, every input has to be the same length, so we need to set some length after which we will cutoff the rest of the review. (We will also need to pad the shorter reviews with zeros to make them the same length).

In [3]:
cutoff = 500

In [4]:
from keras.preprocessing import sequence
x_train_padded = sequence.pad_sequences(x_train, maxlen=cutoff)
x_test_padded = sequence.pad_sequences(x_test, maxlen=cutoff)

## Classification

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

Define our model.

Unlike last time, when we used convolutional layers, we're going to use an LSTM, a special type of recurrent network. You can also try using [other recurrent layers](https://keras.io/layers/recurrent/) if you want.

Using recurrent networks means that rather than seeing these reviews as one input happening all at one, with the convolutional layers taking into account which words are next to each other, we are going to see them as a sequence of inputs, with one word occurring at each timestep.

In [11]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(imdb.get_word_index())+3, output_dim=100, input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
lstm_model.add(LSTM(units=32, return_sequences=True))
lstm_model.add(LSTM(units=32))
lstm_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

Train the model. __This takes awhile. You might not want to re-run it.__

In [12]:
lstm_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

Epoch 1/1
25000/25000 [==============================] - 364s 15ms/step - loss: 0.3979 - binary_accuracy: 0.8148


Assess the model. __This takes awhile. You might not want to re-run it.__

In [13]:
lstm_scores = lstm_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*lstm_scores))

25000/25000 [==============================] - 106s 4ms/step
loss: 0.34549520801544187 accuracy: 0.85352


# Exercise 1

Experiment with different model configurations from the one above. Try other recurrent layers, different numbers of layers, change some of the defaults. See [Keras Recurrent Layers](https://keras.io/layers/recurrent/)

## Exploring Simple Recurrent Layers

Before we dive into something as complicated as LSTMs, Let's take a deeper look at simple recurrent layer weights.

In [14]:
import numpy
from keras.layers import SimpleRNN

The neurons in the recurrent layer pass their output to the next layer, but also back to themselves. The input shape says that we'll be passing in one-dimensional inputs of unspecified length (the None is what makes it unspecified).

In [15]:
one_unit_SRNN = Sequential()
one_unit_SRNN.add(SimpleRNN(units=1, input_shape=(None, 1), activation='linear', use_bias=False))

In [16]:
one_unit_SRNN_weights = one_unit_SRNN.get_weights()
one_unit_SRNN_weights

[array([[0.5393332]], dtype=float32), array([[-1.]], dtype=float32)]

In [17]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 1
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()

[array([[1.]], dtype=float32), array([[1.]], dtype=float32)]

This passes in a single sample that has three time steps.

In [18]:
one_unit_SRNN.predict(numpy.array([ [[3], [3], [7]] ]))

array([[13.]], dtype=float32)

# Exercise 2
Figure out what the two weights in the one_unit_SRNN model control. Be sure to test your hypothesis thoroughly. Use different weights and different inputs.

Let's try a slightly larger simple recurrent model.

In [19]:
two_unit_SRNN = Sequential()
two_unit_SRNN.add(SimpleRNN(units=2, input_shape=(None, 1), activation='linear', use_bias=False))

In [20]:
two_unit_SRNN_weights = two_unit_SRNN.get_weights()
two_unit_SRNN_weights

[array([[ 0.16501439, -0.781032  ]], dtype=float32),
 array([[ 0.38265708,  0.9238905 ],
        [ 0.9238905 , -0.38265708]], dtype=float32)]

In [21]:
two_unit_SRNN_weights[0][0][0] = 1
two_unit_SRNN_weights[0][0][1] = 1
two_unit_SRNN_weights[1][0][0] = 0
two_unit_SRNN_weights[1][0][1] = 1
two_unit_SRNN_weights[1][1][0] = 0
two_unit_SRNN_weights[1][1][1] = 1
two_unit_SRNN.set_weights(two_unit_SRNN_weights)
two_unit_SRNN.get_weights()

[array([[1., 1.]], dtype=float32), array([[0., 1.],
        [0., 1.]], dtype=float32)]

This passes in a single sample with four time steps.

In [22]:
two_unit_SRNN.predict(numpy.array([ [[3], [3], [7], [5]] ]))

array([[ 5., 31.]], dtype=float32)

# Exercise 3
What do each of the six weights of the two_unit_SRNN control? Again, test out your hypotheses carefully.